In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
home_url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy/mm/dd")

# Show DataFrame
home_df.show()

In [ ]:
# Drop duplicates and incomplete rows, and count
print(home_df.count())
home_df = home_df.dropna()
print(home_df.count())
home_df = home_df.dropDuplicates()
print(home_df.count())

In [ ]:
# Examine the schema
home_df.printSchema()

In [ ]:
# Create review table for sql 
home_review_df = home_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
home_review_df.show(5)

In [ ]:
# Create product table for sql
home_product_df = home_df.select(["product_id", "product_title"]).groupby("product_id")
home_product_df.show(5)

In [ ]:
# Create customer table for sql
home_customer_df = home_df.groupby("customer_id").agg({"customer_count": "count"})
home_customer_df.show(5)

In [ ]:
# Create vine table for sql
home_vine_df = home_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
home_vine_df.show(5)